Importing transfermarkt data from Data World - David Cereijo

In [234]:
import pandas as pd
import os

repo_dir = os.getcwd()  # Directory of the script
transferMKT_dir = os.path.join(repo_dir, 'transferMKT-data')

Global Functions and Variables:

In [235]:
# Global Variables:
my_leagues = ['ES1', 'IT1', 'GR1', 'GB1', 'FR1']
player_cols = ['date_y', 'player_id', 'player_name', 'player_current_club_id', 'market_value_in_eur', 'player_club_domestic_competition_id']

In [236]:
# Function that creates a CSV from a pandas df:
def make_csv(df, dir, file_name):
    file_path = os.path.join(dir, f'{file_name}.csv')
    return df.to_csv(file_path, index=True)


# Function that converts a datetime column to soccer season format (ex: 1819):
def calculate_season(date):
    year = date.year
    month = date.month
    if month in range(7,12):
        return ((year - 2000) * 100) + (year - 1999)
    else:
        return ((year - 2001) * 100) + (year - 2000)

Import and clean data using links from data world (DW):

In [237]:
# Create a pandas df for the appearances file:
appearances_df = pd.read_csv('https://query.data.world/s/2t4a5mgcrt7xb32ifpci2wijahs7fq?dws=00000')

# Create a pandas dataframe for the appearances file:
player_val_df = pd.read_csv('https://query.data.world/s/bxh6i5g3kll34aqabzszjbecgdzabm?dws=00000')

# Create a pandas for the clubs file:
clubs_df = pd.read_csv('https://query.data.world/s/4iac2yo5mskcbmy6xnsvahtxe5eakd?dws=00000')

In [244]:
# Use this cell for viewing the DW dataframes:
appearances_df

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
0,-1_172127,-1,172127,3999,3999,NaN,Tyronne,GRP,0,0,0,0,90
1,-1_210988,-1,210988,3999,553,NaN,Christos Eleftheriadis,GRP,0,0,0,0,90
2,-1_231471,-1,231471,3999,3999,NaN,Theofanis Tzandaris,GRP,0,0,0,0,90
3,-1_256819,-1,256819,3999,3999,NaN,Nikolaos Tsoukalos,GRP,0,0,0,0,90
4,-1_281919,-1,281919,3999,3999,NaN,Adam Tzanetopoulos,GRP,0,0,0,0,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166210,3869240_701325,3869240,701325,2503,2503,2023-04-17,Bruno Onyemaechi,PO1,0,0,0,0,90
1166211,3869240_71830,3869240,71830,6912,6912,2023-04-17,Bruno Wilson,PO1,0,0,0,0,90
1166212,3869240_739438,3869240,739438,6912,6912,2023-04-17,Matheus Pereira,PO1,0,0,0,0,75
1166213,3869240_818252,3869240,818252,2503,2503,2023-04-17,Pedro Malheiro,PO1,0,0,0,0,90


In [245]:
# Clean the 3 DW dataframes:

# Date and league filtering:
filt_appearances_df = appearances_df[(appearances_df['competition_id'] == 'GB1') & (appearances_df['date'] >= '2017-07-01')]    # EPL and 17/18 season+ only

# Slice by necessary columns only:
app_cols = ['player_id', 'player_club_id', 'date', 'player_name']
filt_appearances_df = filt_appearances_df[app_cols]


# Date and league filtering:
filt_player_val_df = player_val_df[(player_val_df['player_club_domestic_competition_id'] == 'GB1') & (player_val_df['date'] >= '2017-07-01')]    # EPL and 17/18 season+ only

# Slice by necessary columns only:
val_cols = ['player_id', 'date', 'current_club_id', 'market_value_in_eur', 'player_club_domestic_competition_id']
filt_player_val_df = filt_player_val_df[val_cols]


# Date and league filtering:
filt_clubs_df = clubs_df[clubs_df['domestic_competition_id'] == 'GB1']    # EPL only

# Slice by necessary columns only:
clubs_cols = ['club_id', 'name', 'domestic_competition_id']
filt_clubs_df = filt_clubs_df[clubs_cols]

In [246]:
# Standardize common DW column names and sort on date column before merging:
filt_appearances_df = filt_appearances_df.rename(columns={'competition_id' : 'league_id',
                                                          'player_club_id' : 'team_id'}).sort_values('date', ascending=True)

filt_player_val_df = filt_player_val_df.rename(columns={'current_club_id' : 'team_id',
                                                        'player_club_domestic_competition_id' : 'league_id'}).sort_values('date', ascending=True)

filt_clubs_df = filt_clubs_df.rename(columns={'club_id' : 'team_id',
                                              'name' : 'team',
                                              'domestic_competition_id' : 'league_id'})

In [247]:
filt_appearances_df

,player_id,team_id,date,player_name
414219,10003,1003,2017-08-11,Wes Morgan
414246,94005,11,2017-08-11,Sead Kolasinac
414245,93720,11,2017-08-11,Alexandre Lacazette
414244,82442,11,2017-08-11,Olivier Giroud
414242,73522,1003,2017-08-11,Matty James
...,...,...,...,...
1166125,131505,399,2023-04-17,Rodrigo
1166123,105470,31,2023-04-17,Alisson
1166152,61651,31,2023-04-17,Jordan Henderson
1166137,357119,31,2023-04-17,Ibrahima Konaté


In [248]:
# Create a season column in the appearances and player_val DFs:
dw_dfs = [filt_appearances_df, filt_player_val_df]
for df in dw_dfs:

    # Convert the current date column to date format:
    df['date'] = pd.to_datetime(df['date'])

    # Add the season column by calling the calculate_season() function:
    df['season'] = df['date'].apply(lambda x: calculate_season(x))

    # Drop the date column for testing:
    df.drop('date', axis=1, inplace=True)

# Aggregate the filtered appearances DF to the player-season level and reset index:
agg_appearances_df = (filt_appearances_df
                      .groupby(['season', 'team_id', 'player_name']).agg({'player_id' : 'first'})
                      .reset_index()
                      )
agg_appearances_df['season'].fillna(agg_appearances_df['season'], inplace=True)
agg_appearances_df['team_id'].fillna(agg_appearances_df['team_id'], inplace=True)
agg_appearances_df['player_name'].fillna(agg_appearances_df['player_name'], inplace=True)

# Aggregate the player_val DF to the player-season level:
agg_player_val_df = (filt_player_val_df
                     .groupby(['season', 'team_id', 'player_id',]).agg({'market_value_in_eur' : 'mean',
                                                                    'league_id' : 'first'})
                     .reset_index()
                     )
agg_player_val_df['season'].fillna(agg_player_val_df['season'], inplace=True)
agg_player_val_df['team_id'].fillna(agg_player_val_df['team_id'], inplace=True)
agg_player_val_df['player_id'].fillna(agg_player_val_df['player_id'], inplace=True)

In [252]:
# Use this cell to view the agg DFs:
agg_player_val_df

,season,team_id,player_id,market_value_in_eur,league_id
0,1617,11,30982,750000.0,GB1
1,1617,11,102017,28000000.0,GB1
2,1617,11,121990,1500000.0,GB1
3,1617,11,425306,200000.0,GB1
4,1617,29,4267,1000000.0,GB1
...,...,...,...,...,...
6986,2223,3008,67232,175000.0,GB1
6987,2223,3008,118535,1500000.0,GB1
6988,2223,3008,456111,125000.0,GB1
6989,2223,3008,459873,225000.0,GB1


Attempt an EPL merge of the three DW files:

In [254]:
# Try to merge the two DW dataframes on player_id and season:
first_merge = pd.merge(agg_appearances_df, agg_player_val_df, on=['season', 'team_id', 'player_id'])
make_csv(first_merge, repo_dir, "test_merge")

In [255]:
first_merge

,season,team_id,player_name,player_id,market_value_in_eur,league_id
0,1718,11,Granit Xhaka,111455,4.166667e+07,GB1
1,1718,11,Mohamed Elneny,160438,1.000000e+07,GB1
2,1718,11,Per Mertesacker,6710,1.750000e+06,GB1
3,1718,11,Petr Cech,5658,3.833333e+06,GB1
4,1718,11,Reiss Nelson,340325,1.700000e+07,GB1
...,...,...,...,...,...,...
2346,2223,1237,Pervis Estupiñán,349599,2.000000e+07,GB1
2347,2223,1237,Robert Sánchez,403151,2.850000e+07,GB1
2348,2223,1237,Solly March,209212,1.400000e+07,GB1
2349,2223,1237,Tariq Lamptey,504148,1.500000e+07,GB1


In [260]:
# Add team name column to the first merge using the filt_club_df:
second_merge = pd.merge(first_merge, filt_clubs_df, on=['team_id', 'league_id'])
second_merge

,season,team_id,player_name,player_id,market_value_in_eur,league_id,team
0,1718,11,Granit Xhaka,111455,4.166667e+07,GB1,Arsenal FC
1,1718,11,Mohamed Elneny,160438,1.000000e+07,GB1,Arsenal FC
2,1718,11,Per Mertesacker,6710,1.750000e+06,GB1,Arsenal FC
3,1718,11,Petr Cech,5658,3.833333e+06,GB1,Arsenal FC
4,1718,11,Reiss Nelson,340325,1.700000e+07,GB1,Arsenal FC
...,...,...,...,...,...,...,...
2346,2223,703,Serge Aurier,127032,7.000000e+06,GB1,Nottingham Forest
2347,2223,703,Steve Cook,90836,3.000000e+06,GB1,Nottingham Forest
2348,2223,703,Taiwo Awoniyi,295313,2.000000e+07,GB1,Nottingham Forest
2349,2223,703,Wayne Hennessey,45494,5.000000e+05,GB1,Nottingham Forest


^^^^ LEFT OFF ABOVE THIS MARKDOWN CELL

Merge the three DW dataframes:

In [ ]:
# Merge the appearances and player_val df:
merge1 = pd.merge(appearances_df, player_val_df, on='player_id')

In [ ]:
# Make a copy of merge 1 and slice it to include only the necessary columns:
merge2 = merge1.copy(deep=True)

merge3 = merge2[player_cols]
merge3

Filter and split up the merged data for each league:

In [ ]:
# Filter the merged dataframe on league ids and date:
filter_date = '2017-05-31'

filtered_df = merge3[merge3['player_club_domestic_competition_id'].isin(my_leagues) & (merge3['date_y'] >= filter_date)]

In [ ]:
# Generate a list of player names for each league:
league_players_lists = [list(filtered_df[filtered_df['player_club_domestic_competition_id'] == league]['player_name'].unique()) for league in my_leagues]

# Assign each league to one of the lists:
laliga_players = league_players_lists[0]
serieA_players = league_players_lists[1]
bundesliga_players = league_players_lists[2]
epl_players = league_players_lists[3]
ligue1_players = league_players_lists[4]

In [ ]:
# Create a dataframe for each league using a list comprehension:
league_player_dfs = [filtered_df[filtered_df['player_club_domestic_competition_id'] == league].sort_values('date_y') for league in my_leagues]

In [ ]:
league_player_dfs[0]

Bring in the EPL main merged data file for comparison with the DW file:

In [ ]:
# Import the full EPL data from Merged Data directory:
fbref_epl_merge = pd.read_excel(os.path.join(repo_dir, "Merged Data/ENG-Premier League_full_merge.xlsx"))

In [ ]:
# Generate a list of unique player names from the EPL merge:
fbref_epl_player_names = list(fbref_epl_merge['player'].unique())
print(f'Number of players in fbref EPL player list: {len(fbref_epl_player_names)}\n')

# Generate a list of overlapping player names between the fbref list and DW list:
common_epl_players = [player for player in fbref_epl_player_names if player in epl_players]
print(f'Number of overlapping players in fbref/DW EPL player listS: {len(common_epl_players)}')

Shrink down the DW data by creating a season variable with format "Y1Y2":

Create a merged dataframe for the overlapping EPL players: 

In [ ]:
# Filter the two dataframes to only include overlapping player names, past 2022:
fbref_epl_merge1 = fbref_epl_merge[fbref_epl_merge['player'].isin(common_epl_players)]
fbref_epl_merge1 = fbref_epl_merge1[fbref_epl_merge1['season'] >= 2122].sort_values(by='player', ascending=True)

# Shrinking down the data world dataframe:
dw_epl_df = league_player_dfs[3]

# Common players and date > 2022:
dw_epl_df = dw_epl_df[dw_epl_df['player_name'].isin(common_epl_players)].rename(columns={'player_name' : 'player', 'date_y' : 'date'})
dw_epl_df = dw_epl_df[dw_epl_df['date'] >= '2022-05-31']

# Create a year column:
dw_epl_df['date'] = pd.to_datetime(dw_epl_df['date'])
dw_epl_df['year'] = dw_epl_df['date'].dt.year
dw_epl_df['month'] = dw_epl_df['date'].dt.month

# Create a season column:
dw_epl_df['season'] = dw_epl_df['date'].apply(lambda x: calculate_season(x))

# Group by season-player:
dw_epl_df = (dw_epl_df
             .groupby(['player', 'season']).agg({'player_current_club_id':'first', 'market_value_in_eur':'mean', 'player_club_domestic_competition_id':'first'})
             .reset_index()
             .sort_values(by='player', ascending=True)
             )

dw_epl_df

In [ ]:
# Merge the two sliced dataframes:
dw_fbref_epl_merge = pd.merge(fbref_epl_merge1, dw_epl_df, on=['player', 'season'])
dw_fbref_epl_merge